In [1]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
import torchvision

/home/briancy2/.conda/envs/iwildcam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
META_DIR = "data/metadata/metadata/"
TRAIN_DIR = "data/train/train/"
TEST_DIR = "data/test/test/"

test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))
train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))

#Test images
df_test = pd.DataFrame({'id': [item['id'] for item in test_data['images']],
                                'seq_id': [item['seq_id'] for item in test_data['images']],
                                'file_name': [item['file_name'] for item in test_data['images']],
                                'location': [item['location'] for item in test_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in test_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in test_data['images']]})
#Train images
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['images']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})
# Detection for train test
detections = json.load(open(META_DIR+"iwildcam2022_mdv4_detections.json"))['images']
det_dict = dict()
for detection in tqdm(detections):
    det_dict[detection['file']] = detection['detections']
df_detection = pd.DataFrame({'file': [item['file'] for item in detections],
                                'detections': [item['detections'] for item in detections]})
# Test sequence ids
test_sequence_ids = pd.unique(df_test['seq_id'])
# Train sequence id and count
train_seq_count = pd.read_csv(META_DIR+"train_sequence_counts.csv")

100%|██████████| 261428/261428 [00:00<00:00, 1385974.91it/s]


In [3]:
train_seq_count.

SyntaxError: invalid syntax (1790939137.py, line 1)

In [4]:
!git commit 

,file,detections


In [4]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
for sequence_id in tqdm(test_sequence_ids):
    tracker = Tracker(metric)
    results = []
    img_rows = df_test.loc[df_test.seq_id == sequence_id]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('test/'+img_row['file_name'])]['detections'].to_list()[0]
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections if det['conf'] > 0.80]
#         boxes = np.array([d for d in filtered_det])
#         scores = np.array([d.confidence for d in detections])
        for detection in detections:
            print(detection.tlwh)
            print(detection.confidence)
            print(detection.feature)
        tracker.predict()
        tracker.update(detections)
#         for track in tracker.tracks:
#             if not track.is_confirmed() or track.time_since_update > 1:
#                 continue
#             bbox = track.to_tlwh()
#             results.append([
#                 frame_idx, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])


#     # Store results.
#     f = open(output_file, 'w')
#     for row in results:
#         print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1' % (
#             row[0], row[1], row[2], row[3], row[4], row[5]),file=f)


  0%|          | 0/11028 [00:00<?, ?it/s]


[0.    0.55  0.558 0.438]
0.999
[]
INDEX 0
track []
detect indices [0]
tracks linear []
detections [<deep_sort.deep_sort.detection.Detection object at 0x7ffe76339130>]
HI
[]
[]
[0]
track [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
[0.    0.557 0.684 0.431]
0.999
[]
INDEX 1
track [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
detect indices [0]
tracks linear [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
detections [<deep_sort.deep_sort.detection.Detection object at 0x7ffe76485d60>]
HI
cost matrix [[0.19487798]]
[(0, 0)]
[]
[]
track [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
[0.    0.418 0.712 0.581]
0.999
[]
INDEX 2
track [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
detect indices [0]
tracks linear [<deep_sort.deep_sort.track.Track object at 0x7ffe76339580>]
detections [<deep_sort.deep_sort.detection.Detection object at 0x7ffe76485ee0>]
HI
cost matrix [[0.42654945]]
[(0, 0)]
[]
[]
track [<deep_sort.deep_sort

NameError: name 'frame_idx' is not defined

In [2]:
from deep_sort.application_util import preprocessing
from deep_sort.application_util import visualization
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
metric = nn_matching.NearestNeighborDistanceMetric(
        "cosine", 0.2, 100)
for index, seq_row in tqdm(train_seq_count.iterrows()) :
    tracker = Tracker(metric)
    results = []
    img_rows = df_train.loc[df_train.seq_id == seq_row["seq_id"]]
    for index, img_row in img_rows.iterrows():
        detections = df_detection.loc[df_detection.file == ('test/'+img_row['file_name'])]['detections'].to_list()
        detections = [Detection(tuple(det["bbox"]), det["conf"], []) for det in detections if det['conf'] > 0.80]

NameError: name 'tqdm' is not defined

In [ ]:
!git add .

In [37]:
!git commit -am "created deep_sort_predictions notebook"

[main 7789cd0] created deep_sort_predictions notebook
 3 files changed, 318 insertions(+), 9 deletions(-)
 create mode 100644 .ipynb_checkpoints/deep_sort_predictions-checkpoint.ipynb
 create mode 100644 deep_sort_predictions.ipynb
